In [35]:
import pandas as pd
import numpy as np
from pathlib import Path


In [36]:
BASE_DIR = Path("../data/raw")

ventas_path = BASE_DIR / "ventas_empresa.csv"
clientes_path = BASE_DIR / "clientes.csv"
productos_path = BASE_DIR / "productos.csv"
ventas_df = pd.read_csv(ventas_path)

In [37]:
ventas = pd.read_csv(ventas_path)
clientes = pd.read_csv(clientes_path)
productos = pd.read_csv(productos_path)

ventas.head()


,fecha,cliente_id,producto_id,precio,costo,canal,region,descuento,ingreso,ganancia
0,2024-01-01 00:00:00,1302,P007,508.995016,356.376476,App móvil,Norte,0.088019,464.193722,107.817246
1,2024-01-01 01:00:00,1495,P004,328.595731,188.433835,Web,CDMX,0.151516,278.808163,90.374328
2,2024-01-01 02:00:00,1657,P013,284.011194,203.058373,Marketplace,Norte,0.022284,277.682240,74.623867
3,2024-01-01 03:00:00,1767,P015,1147.736919,779.748163,Marketplace,Occidente,0.002397,1144.986362,365.238199
4,2024-01-01 04:00:00,1973,P011,1106.299843,623.443978,App móvil,Norte,0.004273,1101.572204,478.128226


In [38]:
assert ventas["cliente_id"].isin(clientes["cliente_id"]).all(), \
    "❌ Hay ventas con clientes inexistentes"

assert ventas["producto_id"].isin(productos["producto_id"]).all(), \
    "❌ Hay ventas con productos inexistentes"

print("Integridad referencial OK ✅")


Integridad referencial OK ✅


In [39]:
df = (
    ventas
    .merge(clientes, on="cliente_id", how="left")
    .merge(productos, on="producto_id", how="left")
)

df.head()


,fecha,cliente_id,producto_id,precio,costo,canal,region,descuento,ingreso,ganancia,nombre,apellido,nombre_completo,email,nombre_producto,categoria
0,2024-01-01 00:00:00,1302,P007,508.995016,356.376476,App móvil,Norte,0.088019,464.193722,107.817246,Sandra,Quezada,Sandra Quezada,castroangel@example.org,Silla Pro,Hogar
1,2024-01-01 01:00:00,1495,P004,328.595731,188.433835,Web,CDMX,0.151516,278.808163,90.374328,Paulina,Segura,Paulina Segura,vegacamila@example.com,Audífonos Pro,Electrónica
2,2024-01-01 02:00:00,1657,P013,284.011194,203.058373,Marketplace,Norte,0.022284,277.682240,74.623867,Oswaldo,Rolón,Oswaldo Rolón,natividad42@example.net,Sudadera Pro,Ropa
3,2024-01-01 03:00:00,1767,P015,1147.736919,779.748163,Marketplace,Occidente,0.002397,1144.986362,365.238199,Yuridia,Gollum,Yuridia Gollum,linaressilvano@example.org,Chaqueta Pro,Ropa
4,2024-01-01 04:00:00,1973,P011,1106.299843,623.443978,App móvil,Norte,0.004273,1101.572204,478.128226,Mateo,Cervantes,Mateo Cervantes,lnino@example.com,Playera Pro,Ropa


In [40]:
print("Ventas:", ventas.shape)
print("Clientes:", clientes.shape)
print("Productos:", productos.shape)

ventas.isnull().sum()


Ventas: (5000, 10)
Clientes: (1000, 5)
Productos: (15, 3)


fecha          0
cliente_id     0
producto_id    0
precio         0
costo          0
canal          0
region         0
descuento      0
ingreso        0
ganancia       0
dtype: int64

In [41]:
print("Ventas columnas:")
print(ventas.columns)

print("\nProductos columnas:")
print(productos.columns)

print("\nClientes columnas:")
print(clientes.columns)


Ventas columnas:
Index(['fecha', 'cliente_id', 'producto_id', 'precio', 'costo', 'canal',
       'region', 'descuento', 'ingreso', 'ganancia'],
      dtype='object')

Productos columnas:
Index(['producto_id', 'nombre_producto', 'categoria'], dtype='object')

Clientes columnas:
Index(['cliente_id', 'nombre', 'apellido', 'nombre_completo', 'email'], dtype='object')


In [42]:
df["fecha"] = pd.to_datetime(df["fecha"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   fecha            5000 non-null   datetime64[ns]
 1   cliente_id       5000 non-null   int64         
 2   producto_id      5000 non-null   object        
 3   precio           5000 non-null   float64       
 4   costo            5000 non-null   float64       
 5   canal            5000 non-null   object        
 6   region           5000 non-null   object        
 7   descuento        5000 non-null   float64       
 8   ingreso          5000 non-null   float64       
 9   ganancia         5000 non-null   float64       
 10  nombre           5000 non-null   object        
 11  apellido         5000 non-null   object        
 12  nombre_completo  5000 non-null   object        
 13  email            5000 non-null   object        
 14  nombre_producto  5000 non-null   object 

In [43]:
df = df[df["precio"] > 0]
df = df[df["costo"] >= 0]
df = df[df["precio"] >= df["costo"]]


In [44]:
df["margen_pct"] = df["ganancia"] / df["ingreso"]


In [45]:
ticket_cliente = (
    df.groupby("cliente_id")["ingreso"]
    .sum()
    .reset_index(name="ticket_cliente")
)

df = df.merge(ticket_cliente, on="cliente_id", how="left")


In [46]:
df["mes"] = df["fecha"].dt.month
df["anio"] = df["fecha"].dt.year


In [47]:
df["segmento_cliente"] = pd.qcut(
    df["ticket_cliente"],
    q=3,
    labels=["Bajo", "Medio", "Alto"]
)


In [48]:
OUTPUT_PATH = Path("../data/processed/ventas_limpias.csv")

df.to_csv(OUTPUT_PATH, index=False)

print("Datos limpios guardados ✅")


Datos limpios guardados ✅
